In [ ]:
import os
import nest_asyncio
from llama_index.core import Document, VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.text_embeddings_inference import TextEmbeddingsInference
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core import Settings
from typing import List
from dotenv import load_dotenv
import json
# Apply the monkey patch
from chainlit_app.patches import patch

patch.apply_patch()
nest_asyncio.apply()
load_dotenv(dotenv_path=".env.dev")

In [ ]:
from docx import Document

def extract_tables_as_markdown(docx_path):
    doc = Document(docx_path)
    markdown_tables = []
    for table in doc.tables:
        rows = []
        for row in table.rows:
            cells = [cell.text.strip() for cell in row.cells]
            rows.append("| " + " | ".join(cells) + " |")
        if rows:
            header = rows[0]
            separator = "| " + " | ".join(["---"] * len(table.columns)) + " |"
            markdown_table = "\n".join([header, separator] + rows[1:])
            markdown_tables.append(markdown_table)
    return markdown_tables

In [10]:
from llama_index.readers.file import DocxReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import Document
import glob

# ——————————————
# CONFIGURATION
# ——————————————

# 1) Directory where all your .docx files live
DOCX_FOLDER = "documents/"

# ——————————————
# STEP 1: Discover all .docx files
# ——————————————

all_paths = glob.glob(os.path.join(DOCX_FOLDER, "*.docx"))
print(f"Found {len(all_paths)} .docx file(s):")
for p in all_paths:
    print("  •", p)

# ——————————————
# STEP 2: Load each DOCX and wrap as Document
# ——————————————

reader = DocxReader()
raw_documents = []
for file_path in all_paths:
    # load_data returns a list of in‐memory “page” objects 
    docx_pages = reader.load_data(file_path)
    for page_obj in docx_pages:
        raw_documents.append(
            Document(
                text=page_obj.text,
                metadata={"source": os.path.basename(file_path)}
            )
        )

print(f"Loaded {len(raw_documents)} raw Document(s) from all .docx files.")

# ——————————————
# STEP 3: Chunk each Document semantically
# ——————————————

splitter = SentenceSplitter(chunk_size=1000, chunk_overlap=100)

nodes = []
for doc in raw_documents:
    chunks = splitter.split_text(doc.text)
    for i, chunk in enumerate(chunks):
        nodes.append(
            Document(
                text=chunk,
                metadata={**doc.metadata, "chunk_id": i}
            )
        )

print(f"After splitting, we have {len(nodes)} chunked Documents (nodes).")

# ——————————————
# FINAL: Assign to `documents` so the rest of your pipeline can stay unchanged
# ——————————————

documents = nodes
print(f"Final documents ready for indexing: {len(documents)} nodes.")

# Now you can call your index creation exactly as before:
# from llama_index.core.embeddings import CohereEmbedding  # or whichever embed_model you use
# from llama_index.core.storage import StorageContext
# from llama_index.vector_stores import QdrantVectorStore
# from llama_index import VectorStoreIndex

# Example (adjust embed_model, storage_context, etc. to your configuration):
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents=documents,
#     embed_model=Settings.embed_model,
#     storage_context=storage_context,
# )

Found 15 .docx file(s):
  • documents/FA-G-02_StaffReimbursement_SectionAware.docx
  • documents/อำนาจอนุมัติรายจ่ายสำหรับ Purchase Requisition_แปลงตาราง.docx
  • documents/CPAX-FN-005_ProjectInvestment_SectionAware.docx
  • documents/Trade Supplier Registration and Payment Policy_FA-G-15_15012025_for sign.docx
  • documents/FA-G-02 (T) Staff Expense Reimbursement _15012025 (Chat bot).docx
  • documents/CPAX-FN-005_Project Investment_TH_Final_Narrative_Chatbot.docx
  • documents/FA-G-15_SectionAware_Final_Sectioned.docx
  • documents/FA-G-07 Non-Trade Supplier (Chat bot).docx
  • documents/FA-G-17  Tenant Selection and Debt Collection_Chatbot).docx
  • documents/FA-G-17_SectionAwareChunking.docx
  • documents/ระเบียบปฏิบัติ เรื่อง อำนาจอนุมัติ Level of Authorization_แปลงตาราง.docx
  • documents/FA-G-07_NonTradeSupplier_SectionAware.docx
  • documents/FA-B2B-01_CreditMgmt_SectionAware.docx
  • documents/FA-G-08 อำนาจอนุมัติรายจ่ายสำหรับ Purchase Requisition_แปลงตาราง.docx
  • documents/

## Setup Embedding service

In [11]:
from llama_index.core import StorageContext

In [12]:
# Initialize the Qdrant client
# Initialize the embedding settings
embed_model = TextEmbeddingsInference(
    model_name=os.getenv("EMBED_MODEL_ID"),
    base_url=os.getenv("EMBED_BASE_URL"),
    auth_token=f"Bearer {os.getenv('API_KEY_CHATBOT')}",
    timeout=60,
    embed_batch_size=10,
)

Settings.embed_model = embed_model

## Innitiates VectorStore database (Qdrant)

In [13]:
# Load environment variables
qdrant_api_key = os.getenv("QDRANT_API_KEY")
collection_name = os.getenv("QDRANT_COLLECTION_NAME")

# Your docker-compose maps the gRPC port to 6434 on the host.
qdrant_grpc_port = 6434
qdrant_host = "localhost"

print(f"Attempting to connect to Qdrant gRPC on host: {qdrant_host}, port: {qdrant_grpc_port}")
print(f"Using Qdrant Collection Name: {collection_name}")

if not collection_name:
    raise ValueError("QDRANT_COLLECTION_NAME environment variable is not set")

# Correctly initialize the client to use the mapped gRPC port
# AND explicitly disable HTTPS/TLS to match the server configuration.
client = QdrantClient(
    host=qdrant_host,
    grpc_port=qdrant_grpc_port,
    api_key=qdrant_api_key,
    prefer_grpc=True,
    https=False # <--- THIS IS THE FIX
)


# delete collection if it exists
if client.collection_exists(collection_name):
    print(f"Deleting existing collection: {collection_name}")
    client.delete_collection(collection_name)
else:
    print(f"Collection {collection_name} does not exist, skipping deletion.")
    

# create our vector store with hybrid indexing enabled
vector_store = QdrantVectorStore(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    client=client,
    enable_hybrid=True,
    batch_size=20,
    prefer_grpc=True,
)


Attempting to connect to Qdrant gRPC on host: localhost, port: 6434
Using Qdrant Collection Name: FAQ_DATA


/var/folders/z5/qxv1ygl57v34c8nqkcj1m3200000gn/T/ipykernel_62644/149784803.py:17: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(


Collection FAQ_DATA does not exist, skipping deletion.


## Start embedding process.... into vector database

In [14]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents=documents, embed_model=embed_model, storage_context=storage_context,
)

2025-06-28 16:49:56,912 - INFO - HTTP Request: POST https://api.cpxis.global.lotuss.org/embedding/BAAI/bge-m3/embed "HTTP/1.1 200 OK"


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Try to retrive relavent nodes with question.

In [ ]:
search_query_retriever = index.as_retriever()

search_query_retrieved_nodes = search_query_retriever.retrieve(
"Do all Walmart locations offer scan & go?"
)

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node
for n in search_query_retrieved_nodes:
    display_source_node(n, source_length=2000)